In [1]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as sm
import sys
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LassoLars
from sklearn.ensemble import GradientBoostingRegressor

In [2]:
test = pd.read_csv("/home/sir/Documents/mercedes/test.csv") # test has no y value, y comes from train split
train = pd.read_csv("/home/sir/Documents/mercedes/train.csv")

uniq = {}
#dummyvariables
for colname in ['X0', 'X1', 'X2','X3','X4','X5','X6','X8']:
    uniq[colname] = train[colname].unique()
for colname in ['X0', 'X1', 'X2','X3','X4','X5','X6','X8']:
    train[colname] = train[colname].apply(lambda x: np.where(uniq[colname] == x)[0][0])

In [3]:
train.head()

,ID,y,X0,X1,X2,X3,X4,X5,X6,X8,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,0,130.81,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,6,88.53,0,1,1,1,0,1,1,0,...,1,0,0,0,0,0,0,0,0,0
2,7,76.26,1,2,2,2,0,2,0,1,...,0,0,0,0,0,0,1,0,0,0
3,9,80.62,1,1,2,3,0,2,1,2,...,0,0,0,0,0,0,0,0,0,0
4,13,78.02,1,0,2,3,0,3,2,3,...,0,0,0,0,0,0,0,0,0,0


In [44]:
data = np.asarray(train)
X,y = data[:, 2:], data[:, 1]
lr = LinearRegression()
lr.fit(X, y)
lr.coef_ # which factors are most importante
lr.intercept_
lr.score(X,y)

0.59196992348988053

In [28]:
data = np.asarray(train)
X,y = data[:, 2:], data[:, 1]
lr = LassoLars(alpha=0.01)
lr.fit(X, y)
lr.coef_ # which factors are most importante
lr.intercept_
lr.score(X,y)

/usr/local/lib/python3.5/dist-packages/sklearn/linear_model/least_angle.py:309: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 3 iterations, i.e. alpha=4.682e-02, with an active set of 3 regressors, and the smallest cholesky pivot element being 1.054e-08
  ConvergenceWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/linear_model/least_angle.py:309: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 4 iterations, i.e. alpha=2.706e-02, with an active set of 4 regressors, and the smallest cholesky pivot element being 1.054e-08
  ConvergenceWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/linear_model/least_angle.py:309: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 4 iterations, i.e. alpha=2.706e-02, with an active set of 4 regressors, and the smallest cholesky pivot element being 2.220e-16
  ConvergenceWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/linear_m

0.54101214531137609

In [6]:
data = np.asarray(train)
X,y = data[:, 2:], data[:, 1]
lr = SGDRegressor()
lr.fit(X, y)
lr.score(X,y)

NameError: name 'SGDRegressor' is not defined

In [29]:
params = {'n_estimators': 500, 'max_depth': 4, 'min_samples_split': 2,
          'learning_rate': 0.01, 'loss': 'ls'}
gbr = GradientBoostingRegressor(**params)
gbr.fit(X,y)
gbr.score(X,y)


0.62448863253826614

In [9]:
import xgboost as xgb

In [38]:
from sklearn.datasets import load_boston
from sklearn.model_selection import KFold, train_test_split, GridSearchCV
from sklearn.metrics import r2_score, mean_squared_error
import sys
print("My shit: regression")
kf = KFold(n_splits=10, shuffle=True)
for train_index, test_index in kf.split(X):
    xgb_model = xgb.XGBRegressor().fit(X[train_index],y[train_index])
    predictions = xgb_model.predict(X[test_index])
    actuals = y[test_index]
    sys.stdout.write('\r' + str(r2_score(actuals, predictions)))
    sys.stdout.flush()

My shit: regression
0.642482844435

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy
%matplotlib inline
from keras import backend as K

def r2_keras(y_true, y_pred):
    SS_res =  K.sum(K.square(y_true - y_pred)) 
    SS_tot = K.sum(K.square(y_true - K.mean(y_true))) 
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

#Red data from csv file for training and validation data
X_train, X_test, y_train, y_test = X[:int(len(x)*.8)], y[:int(len(x)*.8)], X[int(len(x)*.8):], y[int(len(x)*.8):]



# create model
model = Sequential()
model.add(Dense(20, activation="selu", kernel_initializer="uniform", input_shape=(376,)))
model.add(Dense(1, activation="linear", kernel_initializer="uniform"))

# Compile model
model.compile(loss='mse', optimizer='adam', metrics=[r2_keras])

# Fit the model
model.fit(X_train, X_test, epochs=5000, batch_size=10,  verbose=2)

# Calculate predictions
PredTestSet = model.predict(X_train)
PredValSet = model.predict(y_train)

# Save predictions
numpy.savetxt("trainresults.csv", PredTestSet, delimiter=",")
numpy.savetxt("valresults.csv", PredValSet, delimiter=",")
                                                    
#Plot actual vs predition for training set
TestResults = numpy.genfromtxt("trainresults.csv", delimiter=",")
plt.plot(Y1,TestResults,'ro')

#Compute R-Square value for training set
TestR2Value = r2_score(Y1,TestResults)
print("Training Set R-Square=", TestR2Value)

#Plot actual vs predition for validation set
ValResults = numpy.genfromtxt("valresults.csv", delimiter=",")
plt.plot(Y2,ValResults,'ro')

#Compute R-Square value for validation set
ValR2Value = r2_score(Y2,ValResults)
print("Validation Set R-Square=",ValR2Value)

Epoch 1/5000
0s - loss: 2599.8782 - r2_keras: -2.3551e+01
Epoch 2/5000
0s - loss: 244.8220 - r2_keras: -1.0415e+00
Epoch 3/5000
0s - loss: 187.3794 - r2_keras: -4.7985e-01
Epoch 4/5000
0s - loss: 144.7063 - r2_keras: -1.1056e-01
Epoch 5/5000
0s - loss: 117.1361 - r2_keras: 0.1377
Epoch 6/5000
0s - loss: 100.1929 - r2_keras: 0.3027
Epoch 7/5000
0s - loss: 90.7715 - r2_keras: 0.3816
Epoch 8/5000
0s - loss: 85.3535 - r2_keras: 0.4069
Epoch 9/5000
0s - loss: 81.7159 - r2_keras: 0.4364
Epoch 10/5000
0s - loss: 78.9697 - r2_keras: 0.4802
Epoch 11/5000
0s - loss: 77.1940 - r2_keras: 0.4991
Epoch 12/5000
0s - loss: 75.7590 - r2_keras: 0.5038
Epoch 13/5000
0s - loss: 75.7940 - r2_keras: 0.5108
Epoch 14/5000
0s - loss: 74.5533 - r2_keras: 0.5280
Epoch 15/5000
0s - loss: 73.5567 - r2_keras: 0.5300
Epoch 16/5000
0s - loss: 73.1848 - r2_keras: 0.5137
Epoch 17/5000
0s - loss: 72.6802 - r2_keras: 0.5326
Epoch 18/5000
0s - loss: 73.1523 - r2_keras: 0.5354
Epoch 19/5000
0s - loss: 71.9781 - r2_keras: 0

0s - loss: 56.2937 - r2_keras: 0.6337
Epoch 158/5000
0s - loss: 56.6381 - r2_keras: 0.6048
Epoch 159/5000
0s - loss: 56.4600 - r2_keras: 0.6177
Epoch 160/5000
0s - loss: 57.0915 - r2_keras: 0.6144
Epoch 161/5000
0s - loss: 56.1547 - r2_keras: 0.6268
Epoch 162/5000
0s - loss: 55.8681 - r2_keras: 0.6342
Epoch 163/5000
0s - loss: 56.7400 - r2_keras: 0.6239
Epoch 164/5000
0s - loss: 56.2169 - r2_keras: 0.6307
Epoch 165/5000
0s - loss: 55.9783 - r2_keras: 0.6279
Epoch 166/5000
0s - loss: 56.0702 - r2_keras: 0.6271
Epoch 167/5000
0s - loss: 56.0851 - r2_keras: 0.6244
Epoch 168/5000
0s - loss: 55.8777 - r2_keras: 0.6174
Epoch 169/5000
0s - loss: 56.2992 - r2_keras: 0.6264
Epoch 170/5000
0s - loss: 56.1243 - r2_keras: 0.6243
Epoch 171/5000
0s - loss: 56.0969 - r2_keras: 0.6262
Epoch 172/5000
0s - loss: 56.4841 - r2_keras: 0.6219
Epoch 173/5000
0s - loss: 56.0326 - r2_keras: 0.6263
Epoch 174/5000
0s - loss: 56.0319 - r2_keras: 0.6225
Epoch 175/5000
0s - loss: 55.6460 - r2_keras: 0.6313
Epoch 17

0s - loss: 53.1678 - r2_keras: 0.6299
Epoch 313/5000
0s - loss: 52.3166 - r2_keras: 0.6576
Epoch 314/5000
0s - loss: 52.9584 - r2_keras: 0.6452
Epoch 315/5000
0s - loss: 52.7966 - r2_keras: 0.6540
Epoch 316/5000
0s - loss: 52.3562 - r2_keras: 0.6180
Epoch 317/5000
0s - loss: 52.5421 - r2_keras: 0.6450
Epoch 318/5000
0s - loss: 53.3685 - r2_keras: 0.6189
Epoch 319/5000
0s - loss: 52.6863 - r2_keras: 0.6461
Epoch 320/5000
0s - loss: 52.6111 - r2_keras: 0.6444
Epoch 321/5000
0s - loss: 52.9702 - r2_keras: 0.6364
Epoch 322/5000
0s - loss: 52.6092 - r2_keras: 0.6394
Epoch 323/5000
0s - loss: 52.8106 - r2_keras: 0.6457
Epoch 324/5000
0s - loss: 52.2880 - r2_keras: 0.6505
Epoch 325/5000
0s - loss: 52.8425 - r2_keras: 0.6463
Epoch 326/5000
0s - loss: 52.8498 - r2_keras: 0.6350
Epoch 327/5000
0s - loss: 52.7590 - r2_keras: 0.6507
Epoch 328/5000
0s - loss: 53.2357 - r2_keras: 0.6340
Epoch 329/5000
0s - loss: 52.9973 - r2_keras: 0.6392
Epoch 330/5000
0s - loss: 52.6310 - r2_keras: 0.6430
Epoch 33

0s - loss: 50.8858 - r2_keras: 0.6528
Epoch 468/5000
0s - loss: 51.3466 - r2_keras: 0.6491
Epoch 469/5000
0s - loss: 51.0666 - r2_keras: 0.6491
Epoch 470/5000
0s - loss: 51.0901 - r2_keras: 0.6527
Epoch 471/5000
0s - loss: 50.8157 - r2_keras: 0.6521
Epoch 472/5000
0s - loss: 51.9704 - r2_keras: 0.6392
Epoch 473/5000
0s - loss: 51.4014 - r2_keras: 0.6499
Epoch 474/5000
0s - loss: 51.0426 - r2_keras: 0.6548
Epoch 475/5000
0s - loss: 51.1635 - r2_keras: 0.6425
Epoch 476/5000
0s - loss: 51.3079 - r2_keras: 0.6532
Epoch 477/5000
0s - loss: 51.7017 - r2_keras: 0.6443
Epoch 478/5000
0s - loss: 51.2193 - r2_keras: 0.6538
Epoch 479/5000
0s - loss: 51.1876 - r2_keras: 0.6527
Epoch 480/5000
0s - loss: 51.1645 - r2_keras: 0.6408
Epoch 481/5000
0s - loss: 51.5061 - r2_keras: 0.6536
Epoch 482/5000
0s - loss: 51.0768 - r2_keras: 0.6493
Epoch 483/5000
0s - loss: 51.2791 - r2_keras: 0.6523
Epoch 484/5000
0s - loss: 51.0025 - r2_keras: 0.6450
Epoch 485/5000
0s - loss: 51.3922 - r2_keras: 0.6399
Epoch 48

0s - loss: 50.7500 - r2_keras: 0.6500
Epoch 623/5000
0s - loss: 49.8253 - r2_keras: 0.6523
Epoch 624/5000
0s - loss: 50.3026 - r2_keras: 0.6581
Epoch 625/5000
0s - loss: 50.2803 - r2_keras: 0.6548
Epoch 626/5000
0s - loss: 50.0580 - r2_keras: 0.6590
Epoch 627/5000
0s - loss: 50.5632 - r2_keras: 0.6497
Epoch 628/5000
0s - loss: 50.5884 - r2_keras: 0.6562
Epoch 629/5000
0s - loss: 50.3935 - r2_keras: 0.6541
Epoch 630/5000
0s - loss: 49.9165 - r2_keras: 0.6631
Epoch 631/5000
0s - loss: 49.9703 - r2_keras: 0.6598
Epoch 632/5000
0s - loss: 50.7063 - r2_keras: 0.6529
Epoch 633/5000
0s - loss: 50.0259 - r2_keras: 0.6388
Epoch 634/5000
0s - loss: 49.9996 - r2_keras: 0.6656
Epoch 635/5000
0s - loss: 49.9104 - r2_keras: 0.6579
Epoch 636/5000
0s - loss: 50.0612 - r2_keras: 0.6524
Epoch 637/5000
0s - loss: 50.0935 - r2_keras: 0.6604
Epoch 638/5000
0s - loss: 50.7739 - r2_keras: 0.6386
Epoch 639/5000
0s - loss: 50.4054 - r2_keras: 0.6598
Epoch 640/5000
0s - loss: 49.9625 - r2_keras: 0.6625
Epoch 64

...

0s - loss: 33.3991 - r2_keras: 0.7484
Epoch 2910/5000
0s - loss: 33.8783 - r2_keras: 0.7423
Epoch 2911/5000
0s - loss: 33.9889 - r2_keras: 0.7469
Epoch 2912/5000
0s - loss: 33.9119 - r2_keras: 0.7503
Epoch 2913/5000
0s - loss: 34.0369 - r2_keras: 0.7470
Epoch 2914/5000
0s - loss: 33.8270 - r2_keras: 0.7509
Epoch 2915/5000
0s - loss: 33.9073 - r2_keras: 0.7536
Epoch 2916/5000
0s - loss: 34.1643 - r2_keras: 0.7425
Epoch 2917/5000
0s - loss: 33.8549 - r2_keras: 0.7449
Epoch 2918/5000
0s - loss: 33.6387 - r2_keras: 0.7428
Epoch 2919/5000
0s - loss: 33.7737 - r2_keras: 0.7448
Epoch 2920/5000
0s - loss: 33.7589 - r2_keras: 0.7495
Epoch 2921/5000
0s - loss: 34.0770 - r2_keras: 0.7454
Epoch 2922/5000
0s - loss: 33.9017 - r2_keras: 0.7443
Epoch 2923/5000
0s - loss: 34.2734 - r2_keras: 0.7503
Epoch 2924/5000
0s - loss: 33.9337 - r2_keras: 0.7472
Epoch 2925/5000
0s - loss: 33.6263 - r2_keras: 0.7453
Epoch 2926/5000
0s - loss: 33.8932 - r2_keras: 0.7449
Epoch 2927/5000
0s - loss: 33.8381 - r2_kera

0s - loss: 33.5554 - r2_keras: 0.7502
Epoch 3062/5000
0s - loss: 33.4514 - r2_keras: 0.7531
Epoch 3063/5000
0s - loss: 33.1934 - r2_keras: 0.7498
Epoch 3064/5000
0s - loss: 33.1069 - r2_keras: 0.7479
Epoch 3065/5000
0s - loss: 33.0675 - r2_keras: 0.7488
Epoch 3066/5000
0s - loss: 33.1131 - r2_keras: 0.7536
Epoch 3067/5000
0s - loss: 33.1972 - r2_keras: 0.7578
Epoch 3068/5000
0s - loss: 33.0879 - r2_keras: 0.7541
Epoch 3069/5000
0s - loss: 33.5149 - r2_keras: 0.7388
Epoch 3070/5000
0s - loss: 33.4105 - r2_keras: 0.7547
Epoch 3071/5000
0s - loss: 33.4935 - r2_keras: 0.7496
Epoch 3072/5000
0s - loss: 33.1873 - r2_keras: 0.7482
Epoch 3073/5000
0s - loss: 33.6145 - r2_keras: 0.7470
Epoch 3074/5000
0s - loss: 33.5906 - r2_keras: 0.7454
Epoch 3075/5000
0s - loss: 33.3533 - r2_keras: 0.7441
Epoch 3076/5000
0s - loss: 32.9133 - r2_keras: 0.7502
Epoch 3077/5000
0s - loss: 32.8389 - r2_keras: 0.7638
Epoch 3078/5000
0s - loss: 33.5273 - r2_keras: 0.7467
Epoch 3079/5000
0s - loss: 33.5441 - r2_kera

0s - loss: 32.5649 - r2_keras: 0.7415
Epoch 3214/5000
0s - loss: 32.4266 - r2_keras: 0.7554
Epoch 3215/5000
0s - loss: 32.4323 - r2_keras: 0.7515
Epoch 3216/5000
0s - loss: 32.3212 - r2_keras: 0.7611
Epoch 3217/5000
0s - loss: 32.0800 - r2_keras: 0.7545
Epoch 3218/5000
0s - loss: 32.3265 - r2_keras: 0.7548
Epoch 3219/5000
0s - loss: 32.2746 - r2_keras: 0.7583
Epoch 3220/5000
0s - loss: 31.8306 - r2_keras: 0.7635
Epoch 3221/5000
0s - loss: 32.0690 - r2_keras: 0.7579
Epoch 3222/5000
0s - loss: 32.0349 - r2_keras: 0.7608
Epoch 3223/5000
0s - loss: 32.5795 - r2_keras: 0.7547
Epoch 3224/5000
0s - loss: 32.4840 - r2_keras: 0.7563
Epoch 3225/5000
0s - loss: 31.9842 - r2_keras: 0.7595
Epoch 3226/5000
0s - loss: 32.2377 - r2_keras: 0.7616
Epoch 3227/5000
0s - loss: 32.1182 - r2_keras: 0.7623
Epoch 3228/5000
0s - loss: 31.9342 - r2_keras: 0.7591
Epoch 3229/5000
0s - loss: 32.1206 - r2_keras: 0.7558
Epoch 3230/5000
0s - loss: 32.2615 - r2_keras: 0.7589
Epoch 3231/5000
0s - loss: 32.1268 - r2_kera

0s - loss: 31.1770 - r2_keras: 0.7699
Epoch 3366/5000
0s - loss: 30.9990 - r2_keras: 0.7678
Epoch 3367/5000
0s - loss: 31.0578 - r2_keras: 0.7563
Epoch 3368/5000
0s - loss: 31.2012 - r2_keras: 0.7625
Epoch 3369/5000
0s - loss: 30.8171 - r2_keras: 0.7661
Epoch 3370/5000
0s - loss: 30.6027 - r2_keras: 0.7732
Epoch 3371/5000
0s - loss: 30.8465 - r2_keras: 0.7636
Epoch 3372/5000
0s - loss: 31.4149 - r2_keras: 0.7626
Epoch 3373/5000
0s - loss: 31.2783 - r2_keras: 0.7571
Epoch 3374/5000
0s - loss: 30.9321 - r2_keras: 0.7670
Epoch 3375/5000
0s - loss: 30.9792 - r2_keras: 0.7614
Epoch 3376/5000
0s - loss: 30.8785 - r2_keras: 0.7640
Epoch 3377/5000
0s - loss: 31.3302 - r2_keras: 0.7626
Epoch 3378/5000
0s - loss: 31.1695 - r2_keras: 0.7730
Epoch 3379/5000
0s - loss: 30.5440 - r2_keras: 0.7732
Epoch 3380/5000
0s - loss: 31.1745 - r2_keras: 0.7654
Epoch 3381/5000
0s - loss: 31.0561 - r2_keras: 0.7744
Epoch 3382/5000
0s - loss: 30.9291 - r2_keras: 0.7587
Epoch 3383/5000
0s - loss: 30.6810 - r2_kera

0s - loss: 30.2839 - r2_keras: 0.7791
Epoch 3518/5000
0s - loss: 30.1710 - r2_keras: 0.7733
Epoch 3519/5000
0s - loss: 30.2721 - r2_keras: 0.7744
Epoch 3520/5000
0s - loss: 30.1501 - r2_keras: 0.7722
Epoch 3521/5000
0s - loss: 30.3009 - r2_keras: 0.7742
Epoch 3522/5000
0s - loss: 30.2731 - r2_keras: 0.7699
Epoch 3523/5000
0s - loss: 30.2328 - r2_keras: 0.7725
Epoch 3524/5000
0s - loss: 29.9547 - r2_keras: 0.7834
Epoch 3525/5000
0s - loss: 30.0154 - r2_keras: 0.7700
Epoch 3526/5000
0s - loss: 30.1110 - r2_keras: 0.7745
Epoch 3527/5000
0s - loss: 29.9570 - r2_keras: 0.7787
Epoch 3528/5000
0s - loss: 29.9733 - r2_keras: 0.7725
Epoch 3529/5000
0s - loss: 29.9135 - r2_keras: 0.7750
Epoch 3530/5000
0s - loss: 30.4555 - r2_keras: 0.7702
Epoch 3531/5000
0s - loss: 29.9084 - r2_keras: 0.7788
Epoch 3532/5000
0s - loss: 30.3947 - r2_keras: 0.7739
Epoch 3533/5000
0s - loss: 29.9562 - r2_keras: 0.7687
Epoch 3534/5000
0s - loss: 30.3909 - r2_keras: 0.7668
Epoch 3535/5000
0s - loss: 30.0616 - r2_kera

In [55]:
X[0].shape


(376,)